In [4]:
from scipy.spatial.distance import cdist
from scipy.stats import mode
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from itertools import permutations, combinations

from numpy.random import default_rng
from numpy.linalg import norm
from scipy.special import softmax
from collections import Counter
from pathlib import Path
from sklearn.cluster import KMeans

def normalize(x):
    return (x - x.mean(axis=0))/x.std(axis=0)

def get_features(path, norm=True):
    data = np.load(path)
    ss, st = data['s'], data['t']
    ssx, ssy = ss[:, :-1], ss[:, -1]
    stx, sty = st[:, :-1], st[:, -1]
    if norm:
        return normalize(ssx), ssy, normalize(stx), sty
    return ssx, ssy, stx, sty

def prototype_classifier(X, C, th=1):
    dist = cdist(C, X)
    prob = softmax(-dist, axis=0)
#     prob = softmax(1/(1+dist), axis=0)
    if th == 1:
        return prob.argmax(axis=0), prob
    return prob.argsort(axis=0)[-th:][::-1, :], prob

# def prototype_classifier(X, C):
#     dist = cdist(C, X)
#     prob = softmax(-dist, axis=0)
#     return prob.T

def masked_prototypical_classifier(X, C, ratio=0.3, seed=2437, th=2):
    rng = np.random.default_rng(seed)
    masked_idx = ~rng.binomial(1, ratio, X.shape[1]).astype(bool)
    return prototype_classifier(X[:, masked_idx], C[:, masked_idx], th=th)

In [6]:
avg = 0
for s, t in permutations(range(4), 2):
    s_path = f'./OfficeHome/image_net/s{s}_t{t}.npz'
    sx, sy, tx, ty = get_features(s_path, norm=False)
    sc = np.stack([sx[sy == i].mean(axis=0) for i in range(65)])  
    pred, _ = prototype_classifier(tx, sc)
    
    
#     mean = np.stack([np.average(stx, axis=0, weights=prob[i]) for i in range(65)])
    
#     pred, _ = prototype_classifier(stx, mean)
    
#     score = (pred == sty).mean()
#     print('-'*10, f'source {s}, target {t}', '-'*10)
#     print(score)
#     avg += score
    break
print('Avg acc.:', avg/12)

[17 62  0 ... 56 64 64]
Avg acc.: 0.0
